In [ ]:
# Define variables
planting_date = as.POSIXct("2013-11-26")

In [ ]:
# Read data from file
vis.data = read.table(file="./data/vis_snapshots_nocorrect.csv", sep=",", header=TRUE)

In [ ]:
# Add water treatment column coded in barcodes
vis.data$treatment <- NA
vis.data$treatment[grep("AA", vis.data$plant_id)] <- 100
vis.data$treatment[grep("AB", vis.data$plant_id)] <- 0
vis.data$treatment[grep("AC", vis.data$plant_id)] <- 16
vis.data$treatment[grep("AD", vis.data$plant_id)] <- 33
vis.data$treatment[grep("AE", vis.data$plant_id)] <- 66

In [ ]:
# Add plant genotype column coded in barcodes
vis.data$genotype <- NA
vis.data$genotype[grep("p1", vis.data$plant_id)] <- 'A10'
vis.data$genotype[grep("p2", vis.data$plant_id)] <- 'B100'
vis.data$genotype[grep("r1", vis.data$plant_id)] <- 'R20'
vis.data$genotype[grep("r2", vis.data$plant_id)] <- 'R70'
vis.data$genotype[grep("r3", vis.data$plant_id)] <- 'R98'
vis.data$genotype[grep("r4", vis.data$plant_id)] <- 'R102'
vis.data$genotype[grep("r5", vis.data$plant_id)] <- 'R128'
vis.data$genotype[grep("r6", vis.data$plant_id)] <- 'R133'
vis.data$genotype[grep("r7", vis.data$plant_id)] <- 'R161'
vis.data$genotype[grep("r8", vis.data$plant_id)] <- 'R187'

In [ ]:
# Add genotype x treatment group column
vis.data$group = paste(vis.data$genotype,'-',vis.data$treatment,sep='')

In [ ]:
# Add calendar-time data column using the Unix-time data
vis.data$date = as.POSIXct(vis.data$datetime, origin = "1970-01-01")

In [ ]:
# Calculate days after planting from planting data
vis.data$dap = as.numeric(vis.data$date - planting_date)

In [ ]:
# Soil volume water content
# Use linear regression to create a simple model for using water volume to predict soil volume water content
vwc.data = read.table(file="./data/soil_weight_vwc.txt", sep="\t", header=TRUE)
# Create a linear model for water volume and volume water content. Water volumes >= 260 appear to have
# saturated the soil water carrying capacity, so remove them from the model.
vwc.lm = lm(vwc_wet ~ water_vol, data=vwc.data[vwc.data$water_vol < 260,])
# Predict volume water contents for the water treatment groups
treatment.water = data.frame(water_vol=c(217, 144.5, 72))
treatment.water$vwc = predict(object = vwc.lm, newdata=treatment.water)

In [ ]:
# Convert VIS camera zoom units
# LemnaTec VIS camera zoom units range from 1 to 6000, which correspond to 1 to 6X zoom
zoom.lm = lm(zoom.camera ~ zoom, data=data.frame(zoom=c(1,6000), zoom.camera=c(1,6)))

In [ ]:
# VIS zoom correction
# In this section we define models that are used to convert area and length between camera zoom levels
# to a common scale.
z.data = read.table(file="./data/zoom_calibration_data.txt", sep="\t", header=TRUE)
z.data$px_cm = z.data$length_px / z.data$length_cm
# Convert LemnaTec zoom units to camera zoom units
z.data$zoom.camera = predict(object = zoom.lm, newdata=z.data)
vis.data$zoom = vis.data$sv_zoom
vis.data$sv.zoom.camera = predict(object = zoom.lm, newdata=vis.data)
vis.data$zoom = vis.data$tv_zoom
vis.data$tv.zoom.camera = predict(object = zoom.lm, newdata=vis.data)

In [ ]:
# Zoom correction for area (exponential non-linear model)
area.coef = coef(nls(log(rel_area) ~ log(a * exp(b * zoom.camera)), z.data, start = c(a = 1, b = 0.01)))
area.coef = data.frame(a=area.coef[1], b=area.coef[2])
area.nls = nls(rel_area ~ a * exp(b * zoom.camera), data = z.data, start=c(a=area.coef$a, b=area.coef$b))

In [ ]:
# Zoom correction for length (polynomial non-linear model)
len.poly = lm(px_cm ~ zoom.camera + I(zoom.camera^2), data=z.data[z.data$camera == 'VIS SV',])

In [ ]:
# Correct VIS data for differences in camera zoom
vis.data.zoom = vis.data[,c('plant_id', 'datetime', 'treatment', 'genotype', 
                            'group', 'date', 'dap', 'solidity', 'outlier')]
# Predict relative area zoom correction factors
vis.data$zoom.camera = vis.data$sv.zoom.camera
vis.data$sv_rel_area = predict(object = area.nls, newdata = vis.data)
vis.data$zoom.camera = vis.data$tv.zoom.camera
vis.data$tv_rel_area = predict(object = area.nls, newdata = vis.data)
# Calculate total zoom-corrected side-view and top-view area
vis.data.zoom$sv_area = (vis.data$sv0_area / vis.data$sv_rel_area) + (vis.data$sv90_area / vis.data$sv_rel_area) +
                        (vis.data$sv180_area / vis.data$sv_rel_area) + (vis.data$sv270_area / vis.data$sv_rel_area)

vis.data.zoom$tv_area = vis.data$tv_area / vis.data$tv_rel_area

In [ ]:
# Calculate zoom-corrected lengths
vis.data$zoom.camera = vis.data$sv.zoom.camera
vis.data$px_cm = predict(object = len.poly, newdata=vis.data)
vis.data.zoom$extent_x = vis.data$extent_x / vis.data$px_cm
vis.data.zoom$extent_y = vis.data$extent_y / vis.data$px_cm
vis.data.zoom$height_above_bound = vis.data$height_above_bound / vis.data$px_cm

In [ ]:
# Write cleaned table to file
write.table(vis.data.zoom, file = "vis_snapshots_cleaned.csv", quote = FALSE, sep=",")